# Prototype price chart subplot
- requires database from fx_trader repo
- [Example 5 Dash](https://dash.plot.ly/interactive-graphing) 

### Summary 
Hypothesis:
A trader can inspect the price movement before executing a predicted action: Buy/Sell.

#1 Main Plot: model forecast confidence Vs profit per trade
#2 Sub Plot: 100 days window Vs closing price of stock

---------------------------
Callback 1. 
~~F) create_time_series --> old subplot ~~
F) create_time_series_subplot --> create sub plot
F) update graph --> create main plot

Callback 2.
F) update_x_timeseries --> creates rolling window
    - hoverData --> fixed format 
    - values --> must select range of data
---------------------------


In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql
# import pickle
# import dateutil.parser # to parse ISO date
from datetime import datetime, timedelta

# Plotly
import plotly.graph_objects as go

# Web app
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

database_name = "FX_data/database.db"

conn = sql.connect('FX_data/database.db')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM eda_data_LR ",conn)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'],inplace=True)
conn.close()

In [2]:
df.head(1)

,Target,Prediction,Correct,% Buy,% Sell,Price,Open,High,Low,Close,Prob Buy,Prob Sell,Balance,Profit,comp_0,comp_1,t-SNE_1,t-SNE_2,Cluster,Confidence %
Date,,,,,,,,,,,,,,,,,,,,
2004-03-01,0,1,0,37.25,62.75,0.8034,0.755824,0.762779,0.758515,0.76933,0.364423,0.635577,9903.846154,-96.153846,-0.019401,0.017532,22.393435,-12.548299,5,62.75


In [3]:
# FOR DASH APP
year_options = pd.DatetimeIndex(df.reset_index()['Date']).year #.year

In [90]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    
    # DROP DOWN CLUSTER
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in df['Cluster'].unique()],
                value=7
            ),
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    
    # MAIN GRAPH
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': '2007-03-01'}]}  # custom data is norm
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    
    # SUBPLOT PRICE CHART
    html.Div([
        dcc.Graph(id='x-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    
    # YEAR SLIDER
    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=year_options.min(),
        max=year_options.max(),
        value= 2005,
        marks=  {i: '{}'.format(i) for i in year_options.unique().tolist()},
        #{str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])




In [91]:
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

def update_graph(xaxis_column_name, 
                 year_value):
      
    df["Date_Y"] = pd.DatetimeIndex(df.reset_index()["Date"]).year
    dff = df[ (df["Date_Y"] <= year_value) & (df['Cluster'] == xaxis_column_name)]

    return {
        'data': [dict(
            x=dff['Confidence %'],
            y=dff['Profit'],
            
            text=dff['Cluster'],  # info shown upon hovering
            
            customdata=dff.reset_index()["Date"],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': "Model Confidence (%) for Group: {}".format(xaxis_column_name),
            },
            yaxis={
                'title': "Profit",
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

" REDUNDANT "
# def create_time_series(dff, title):
#     return {
#         'data': [dict(
#             x=dff['Confidence %'],
#             y=dff['Profit'],
#             mode='lines+markers'
#         )],
#         'layout': {
#             'height': 225,
#             'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
#             'annotations': [{
#                 'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
#                 'xref': 'paper', 'yref': 'paper', 'showarrow': False,
#                 'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
#                 'text': title
#             }],
#         }
#     }
    
    
"""
FOR SUBPLOT 
"""
def create_time_series_subplot(dff, title):
    return {
        'data': [dict(
            x=dff['Date'],
            y=dff['Price'],
            mode='lines+markers'
            
        )],
        'layout': {

            'xaxis': {'title': "Rolling Window -100 Days"},
            'yaxis': {'title': "Closing Prices"},
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 
                'y': 0.85, 
                'xanchor': 'left', 
                'yanchor': 'bottom',
                'xref': 'paper', 
                'yref': 'paper', 
                'showarrow': False,
                'align': 'left', 
                'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
    ])
def update_x_timeseries(hoverData, cluster):
    data = hoverData['points'][0]['customdata']
    """
    fix format: hoverData -> datetime -> str
    """
    day = str(pd.to_datetime(data))
    print (day) # 2007-03-01  <- Date
    print (cluster) # 7  <- Cluster

    """
    Get 100 days of history
    """

    try:  dff = df.reset_index().loc[(df.reset_index()['Date'] <= day) & (df.reset_index()['Cluster'] == cluster)][-100:] # max 100 days history
    except:  dff = df.reset_index().loc[(df.reset_index()['Date'] <= day) & (df.reset_index()['Cluster'] == cluster)] # else as much as available 
    
#     dff = df[df['Date'] == country_name]
#     dff = dff[dff['Cluster'] == xaxis_column_name]   # <----- date ???
    title = '<b> Price Chart for forecast: {}</b> <br> Trade Cluster: {}'.format(day[:-8],cluster ) #
    return create_time_series_subplot(dff, title)



if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2020 12:11:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:32] "GET /_favicon.ico?v=1.7.0 HTTP/1.1" 200 -


2007-03-01 00:00:00
7


127.0.0.1 - - [19/Feb/2020 12:11:34] "POST /_dash-update-component HTTP/1.1" 200 -


2005-09-21 00:00:00
7


...

127.0.0.1 - - [19/Feb/2020 12:11:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2020 12:11:41] "POST /_dash-update-component HTTP/1.1" 200 -


2006-01-13 00:00:00
7
2005-05-27 00:00:00
7
2004-03-09 00:00:00
7


127.0.0.1 - - [19/Feb/2020 12:11:41] "POST /_dash-update-component HTTP/1.1" 200 -


2005-08-26 00:00:00
7


127.0.0.1 - - [19/Feb/2020 12:11:41] "POST /_dash-update-component HTTP/1.1" 200 -


=======================================   END   ===============================================